## Import libraries


In [15]:
# Import libraries
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

import sys

from dotenv import load_dotenv
load_dotenv(override=True)
token_huggingface_hub = os.getenv('TOKEN_HUGGINGFACE_HUB')

## Split the doucment into Chunks & Store them in Vector Store


In [16]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("docs\Microsoft SQL Server 2012 T-SQL Fundamentals.pdf")
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./sql_chroma_db")

In [17]:
# only run this once to generate vector store
ingest()

Split 437 documents into 1172 chunks.


In [18]:
from huggingface_hub import login
access_token_read = token_huggingface_hub
access_token_write = token_huggingface_hub
login(token = access_token_read)

## Create a RAG chain that retreives relevent chunks and prepares a response


In [26]:
def rag_chain():
    # 🧠 Inicializa el modelo de lenguaje local (LLM) usando Ollama con el modelo "llama3.2"
    model = ChatOllama(model="llama3.2")

    # 📝 Define el prompt que se le dará al modelo. 
    # Indica que debe responder amigablemente y solo usando el contexto proporcionado.
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context. 
        If you don't know the answer, then reply, No Context available for this question {input}. [/Instructions] </s> 
        [Instructions] Question: {input} 
        Context: {context} 
        Answer: [/Instructions]
        """
    )

    # 🧊 Configura los embeddings para convertir texto a vectores (representaciones numéricas).
    embedding = FastEmbedEmbeddings()

    # 🗃️ Carga (o crea) la base de datos vectorial Chroma desde el disco.
    # Se usará para hacer búsqueda semántica de los documentos.
    vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)

     # 🔍 Crea un recuperador (retriever) a partir de la base vectorial.
    # Busca los documentos más similares al texto de entrada usando umbral de similitud.
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",  # usa similitud + umbral
        search_kwargs={"k": 3, "score_threshold": 0.5}, # trae hasta 3 resultados con score > 0.5
    )

    # 🔗 Crea una cadena de documentos que combina los documentos recuperados con el prompt.
    document_chain = create_stuff_documents_chain(model, prompt)

    # 🔄 Crea la cadena completa de RAG: recupera contexto + responde con el LLM.
    chain = create_retrieval_chain(retriever, document_chain)

    # 🚀 Devuelve la cadena para poder hacer preguntas con ella.
    return chain


In [23]:
def ask(query: str):
    #
    chain = rag_chain()
    # invoke chain
    result = chain.invoke({"input": query})
    # print results
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

## Ask Question to get relevant information from the document

In [24]:
ask("tell me about single-table queries")

Single-table queries are queries that target a single table in a database, rather than multiple tables. They allow you to retrieve specific data from one or more columns within a table, based on certain conditions specified using predicates and operators.

Some key concepts covered in this chapter include:

* Logical query processing: the series of logical phases involved in producing the correct result set
* Predicates and operators: used to filter data and specify conditions for retrieval
* CASE expressions: used to perform conditional logic and return different values based on specific criteria
* NULL marks: used to handle missing or null values in the data
* All-at-once operations: used to retrieve all rows that meet a certain condition, rather than retrieving them individually
* Manipulating character data: using functions like upper() and lower() to manipulate and compare character strings
* Manipulating date and time data: using functions like GETDATE() and DATEADD() to perform 